In [1]:
import torch

In [16]:
import torch
from collections import Counter
import random
import re

file_path = '/Users/jaafarbendriss/First-LLM/dataset/black_hole_wiki.txt'

# Open the file and read its content
with open(file_path, 'r') as file:
    file_content = file.read()

raw_text = file_content

def add_start_end_sentence(text):
    return "start-of-sentence " + text.replace("."," end-of-sentence start-of-sentence")


def remove_citations(text):
    pattern = r"\[\d+\]"
    cleaned_text = re.sub(pattern, "", text)    
    return cleaned_text

def remove_spaces(text):
    pattern = r"\s+"
    cleaned_text = re.sub(pattern, " ", text)    
    return cleaned_text

def preprocess(text: str):
    return remove_spaces(remove_citations(add_start_end_sentence(text).lower().replace(" s ","").replace("\\","").replace("'"," ").replace('"',' ').replace(","," ").replace("\ s"," ").replace("\n"," ")))

preprocessed_text = preprocess(raw_text)

Counter(preprocessed_text.split(" ")).most_common(20)

preprocessed_text_array = preprocessed_text.split(" ")


print(preprocessed_text_array[0:100])

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

tokens = tokenizer(preprocessed_text)

preprocessed_text = preprocessed_text.replace("end-of-sentence start-of-sentence", "end-of-sentence69696969start-of-sentence")
splited_data = preprocessed_text.split("69696969")

train_data =  [(i,sentence) for i, sentence in zip( range(0,len(splited_data)), splited_data  )]
train_iter = iter(train_data)


['start-of-sentence', 'object', 'that', 'has', 'a', 'no-return', 'boundary', 'for', 'other', 'uses', 'see', 'black', 'hole', '(disambiguation)', 'end-of-sentence', 'start-of-sentence', 'direct', 'radio', 'image', 'of', 'a', 'supermassive', 'black', 'hole', 'at', 'the', 'core', 'of', 'messier', '87', 'animated', 'simulation', 'of', 'a', 'schwarzschild', 'black', 'hole', 'with', 'a', 'galaxy', 'passing', 'behind', 'end-of-sentence', 'start-of-sentence', 'around', 'the', 'time', 'of', 'alignment', 'extreme', 'gravitational', 'lensing', 'of', 'the', 'galaxy', 'is', 'observed', 'end-of-sentence', 'start-of-sentence', 'a', 'black', 'hole', 'is', 'a', 'region', 'of', 'spacetime', 'where', 'gravity', 'is', 'so', 'strong', 'that', 'nothing', 'including', 'light', 'and', 'other', 'electromagnetic', 'waves', 'has', 'enough', 'energy', 'to', 'escape', 'it', 'end-of-sentence', 'start-of-sentence', 'einstein', 's', 'theory', 'of', 'general', 'relativity', 'predicts', 'that', 'a', 'sufficiently', 'co

In [17]:
next(train_iter)

(0,
 'start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence')

## Preparing training dataset using sliding window

In [18]:
sentence =  'start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence'

In [19]:
data = sentence.split()

In [25]:
splited_data

['start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence',
 'start-of-sentence direct radio image of a supermassive black hole at the core of messier 87 animated simulation of a schwarzschild black hole with a galaxy passing behind end-of-sentence',
 'start-of-sentence around the time of alignment extreme gravitational lensing of the galaxy is observed end-of-sentence',
 'start-of-sentence a black hole is a region of spacetime where gravity is so strong that nothing including light and other electromagnetic waves has enough energy to escape it end-of-sentence',
 'start-of-sentence einstein s theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole end-of-sentence',
 'start-of-sentence the boundary of no escape is called the event horizon end-of-sentence',
 'start-of-sentence a black hole has a great effect on the fate and circumstances of an object crossing it but it h

In [31]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
vocab(tokens[0:10])

[4, 37, 10, 66, 6, 0, 186, 16, 68, 0]

In [32]:
token_map = vocab.get_stoi()

In [26]:
training_data = []

for sentence in splited_data:
    data=sentence.split()
    for i in range(1,len(data)-1):
        X=data[0:i]
        y = data[i+1]
        #print(i, X)
        #print(y)
        #print()
        training_data.append((X,y))

In [27]:
len(training_data)

11094

In [33]:
training_data[110:115]

[(['start-of-sentence', 'a'], 'hole'),
 (['start-of-sentence', 'a', 'black'], 'has'),
 (['start-of-sentence', 'a', 'black', 'hole'], 'a'),
 (['start-of-sentence', 'a', 'black', 'hole', 'has'], 'great'),
 (['start-of-sentence', 'a', 'black', 'hole', 'has', 'a'], 'effect')]

In [37]:
word_to_token = token_map

def word_to_token_wrapper(word: str) -> int:
    if word not in word_to_token:
        return 0
    return word_to_token[word]

def sentence_to_token(sentence)->int:

    
    return [word_to_token_wrapper(word) for word in sentence]

In [38]:
training_data_tokenized = [ (sentence_to_token(a),word_to_token_wrapper(b) ) for a,b in training_data]

In [39]:
training_data_tokenized[110:115]

[([4, 6], 8),
 ([4, 6, 5], 66),
 ([4, 6, 5, 8], 6),
 ([4, 6, 5, 8, 66], 412),
 ([4, 6, 5, 8, 66, 6], 153)]

training_data_tokenized[110:115]
"""
[([4, 6], 8),
 ([4, 6, 5], 66),
 ([4, 6, 5, 8], 6),
 ([4, 6, 5, 8, 66], 412),
 ([4, 6, 5, 8, 66, 6], 153)]
 """

## training 

In [40]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

class SentenceCompletionDataset(Dataset):
    def __init__(self, data, word_to_index):
        self.data = data
        self.word_to_index = word_to_index
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_sequence, target_word = self.data[idx]
        input_indices = [self.word_to_index[word] for word in input_sequence]
        target_index = self.word_to_index[target_word]
        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_index, dtype=torch.long)

# Example Model (simplified)
class SimpleSeq2Seq(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(SimpleSeq2Seq, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.out = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        output, (hidden, cell) = self.rnn(embedded)
        decoded = self.out(output[:, -1, :])  # Use the last output for prediction
        return decoded


In [41]:
from sklearn.model_selection import train_test_split

# Assuming train_data is a list of sentences
train_sentences, val_sentences = train_test_split(masked_sentences_and_targets_tokens, test_size=0.2, random_state=42)

# Now, apply the masking and preparation logic to both train_sentences and val_sentences

NameError: name 'masked_sentences_and_targets_tokens' is not defined

In [42]:
from torch.utils.data import Dataset, DataLoader
import torch

class TokenizedTextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data
        
    def __len__(self):
        return len(self.tokenized_data)
    
    def __getitem__(self, idx):
        input_seq, target_token = self.tokenized_data[idx]
        return torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_token, dtype=torch.long)

import torch.nn as nn

class SimpleRNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(SimpleRNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.rnn(embedded)
        # Taking the output of the last time step
        output = self.fc(output[:, -1, :])
        return output


In [45]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    tokens, targets = zip(*batch)
    # Pad the sequences to the maximum length in the batch
    tokens_padded = pad_sequence([torch.tensor(seq) for seq in tokens], batch_first=True, padding_value=0)
    targets = torch.tensor(targets, dtype=torch.long)
    return tokens_padded, targets


In [47]:
len(training_data_tokenized)

11094

In [92]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

# Hyperparameters
embedding_dim = 64
hidden_dim = 120
vocab_size = len(word_to_token)

model = SimpleRNNModel(vocab_size, embedding_dim, hidden_dim)
optimizer = Adam(model.parameters(), lr=0.01)
criterion = CrossEntropyLoss()

# DataLoader
dataset = TokenizedTextDataset(training_data_tokenized)
dataloader = DataLoader(dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


/var/folders/l4/bv3z08h91lggkmhcp2n7yzh80000gn/T/ipykernel_65851/763346120.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens_padded = pad_sequence([torch.tensor(seq) for seq in tokens], batch_first=True, padding_value=0)


Epoch 1, Loss: 7.452559471130371
Epoch 2, Loss: 6.41782283782959


In [93]:
def predict_next_token(model, input_seq):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0)  # Add batch dimension
        output = model(input_tensor)
        predicted_token_id = output.argmax(dim=1).item()
    return predicted_token_id


In [94]:
sentence="black hole are the biggest"
sequence = sentence_to_token(sentence)

In [95]:
predicted_token_id = predict_next_token(model, sequence)

In [96]:
predicted_token_id

3

In [97]:
token_to_word = {token: word for word, token in token_map.items()}
predicted_word = token_to_word[predicted_token_id]
predicted_word

'end-of-sentence'

In [98]:
def predict_next_word(start):
    sequence = sentence_to_token(sentence)
    predicted_token_id = predict_next_token(model, sequence)
    token_to_word = {token: word for word, token in token_map.items()}
    predicted_word = token_to_word[predicted_token_id]
    return predicted_word

In [99]:
next_word = "start-of-sentence"
words = []
for _ in range(20):
    words.append(next_word)
    next_word = predict_next_word(next_word)

" ".join(words)

'start-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence end-of-sentence'